In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import pickle

In [3]:
df = pd.read_csv("output.csv")

In [4]:
x= np.array(df["Station Name"])
y= np.array(df["Day"])
z= np.array(df["Time"])
w = np.array(df["Weather"])

In [5]:
print(np.unique(x))

['AIIMS' 'Adarsh Nagar' 'Akshardham' 'Anand Vihar' 'Arjan Garh'
 'Ashok Park Main' 'Azadpur' 'Badarpur' 'Barakhambha Road'
 'Botanical Garden' 'Central Secretariat' 'Chandni Chowk' 'Chawri Bazar'
 'Chhatarpur' 'Civil Lines' 'Delhi Aerocity' 'Dhaula Kuan'
 'Dilshad Garden' 'Dwarka' 'Dwarka Morh' 'Dwarka Sector 10'
 'Dwarka Sector 11' 'Dwarka Sector 12' 'Dwarka Sector 13'
 'Dwarka Sector 14' 'Dwarka Sector 21' 'Dwarka Sector 8' 'Dwarka Sector 9'
 'GTB Nagar' 'Ghitorni' 'Govind Puri' 'Green Park' 'Guru Dronacharya'
 'HUDA City Centre' 'Hauz Khas' 'IFFCO Chowk' 'INA' 'Inderlok'
 'Indira Gandhi International Airport' 'Indraprastha' 'Jahangirpuri'
 'Janakpuri East' 'Janakpuri West' 'Jangpura' 'Jasola Apollo'
 'Jawaharlal Nehru Stadium' 'Jhandewalan' 'Jhilmil' 'Jor Bagh'
 'Kailash Colony' 'Kalkaji Mandir' 'Kanhiya Nagar' 'Karkarduma'
 'Karol Bagh' 'Kashmere Gate' 'Kaushambi' 'Keshav Puram' 'Khan Market'
 'Kirti Nagar' 'Kohat Enclave' 'Lajpat Nagar' 'Laxmi Nagar' 'MG Road'
 'Madipur' 'Malviya 

In [6]:
print(np.unique(y))

['Day' 'FRIDAY' 'MONDAY' 'SATURDAY' 'SUNDAY' 'THURSDAY' 'TUESDAY'
 'WEDNESDAY']


In [7]:
print(np.unique(z))

['10:00' '11:00' '12:00' '1:00' '2:00' '3:00' '4:00' '5:00' '6:00' '7:00'
 'Time']


In [8]:
print(np.unique(w))

['Cool' 'Rainy' 'Sunny' 'ThunderStorm' 'Weather']


In [9]:
def try_int(x):
    try:
        return int(x)
    except ValueError:
        return 

df["Weather"] = pd.Categorical(df["Weather"]).codes
df["Day"] = pd.Categorical(df["Day"]).codes
df["Station Name"] = pd.Categorical(df["Station Name"]).codes
df["Time"] = df["Time"].map(lambda x: try_int(x.split(":")[0]))
df["Population"] = df["Population"].map(lambda x: try_int(x))

In [10]:
x = df[["Station Name", "Day", "Time", "Weather"]].values
y = df[["Population"]].values

In [11]:
x = np.nan_to_num(x)
y = np.nan_to_num(y)
np.any(np.isnan(x))
# norm_x = preprocessing.scale(x)
# norm_y = preprocessing.scale(y)

False

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.35, random_state=100)

In [13]:
RandomForesRegModel = RandomForestRegressor(n_estimators = 6, random_state = 0)
RandomForesRegModel.fit(x_train.reshape(-1,4), y_train.ravel())

RandomForestRegressor(n_estimators=6, random_state=0)

In [14]:
y_pred = RandomForesRegModel.predict(x_test.reshape(-1,4))

In [15]:
y_pred[1:10].reshape(-1,1)

array([[1074.6615639 ],
       [  72.69947665],
       [1099.7677937 ],
       [  33.17528714],
       [1344.13888419],
       [ 108.78733974],
       [ 367.78580553],
       [ 206.57260101],
       [ 858.56593221]])

In [16]:
y_test[1:10].reshape(-1,1)

array([[1127.],
       [  52.],
       [1040.],
       [  16.],
       [1365.],
       [ 136.],
       [ 339.],
       [ 157.],
       [ 867.]])

In [17]:
print('Accuracy : ', r2_score(y_test, y_pred.reshape(-1,1))*100)
print('MSE: ', mean_squared_error(y_test, y_pred.reshape(-1,1)))

Accuracy :  98.19030927379211
MSE:  2214.8806788867537


In [18]:
PKL_filename = "RFR_Model.pkl"

with open(PKL_filename, 'wb') as file:
    pickle.dump(RandomForesRegModel, file)

In [19]:
with open ("RFR_Model.pkl", 'rb') as file:
    Loaded_Model = pickle.load(file)
Loaded_Model
test_input=np.array([1,6,10,2]) # "Station Name[0-137]", "Day[0-7]", "Time[10-8]", "Weather[0-4]"
result = Loaded_Model.predict(test_input.reshape(-1,4))
result[0]

1539.3043803418805

In [20]:
# score = Loaded_Model.score(x_test.reshape(-1,4),y_test)
# score